In [61]:
# User will ask his doubt, based on his experience/field of study, route him to particular advisor (Tech, Marketing, Finance etc) and provide him with path
# with encouragement.

In [62]:
from dotenv import find_dotenv, load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


In [63]:
load_dotenv(find_dotenv())
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.5)


In [64]:
domain_list = ['medical', 'finance', 'tech']

In [65]:
user_query = ("""
                                You are a context extractor, 
                                You are given with a user query and your job  is to classify the domain for the given query.
                                Classification list is as follows: medical, finance, tech
                                You MUST only give single domain name in the output.
                                
                                Here is the user query: {query}
                                Classification :""")

user_query_prompt = ChatPromptTemplate.from_template(user_query)

classified_chain = user_query_prompt | llm

classification = classified_chain.invoke({'query': 'I want to become a cloud architect, in next 3 years, I am coming from Networking background'})


In [ ]:
tech_advisor_prompt = """You are a techincal advisor having more than 15 yeras of consulting and advisory in Technical domain. 
                your answers are very specific, short and encouraging to the users. 
                For a given user's query, provide a appropriate response to the user.
                query: {query}
                    """
tech_advisor = ChatPromptTemplate.from_template(tech_advisor_prompt)

tech_chain = tech_advisor | llm | StrOutputParser()


In [ ]:
financial_advisor_prompt = """You are a financial advisor having more than 15 yeras of consulting and advisory in marketing and other marketing sales domain. 
                your answers are very specific, short and encouraging to the users. 
                For a given user's query, provide a appropriate response to the user.
                query: {query}
                    """
financial_advisor = ChatPromptTemplate.from_template(financial_advisor_prompt)


financial_chain = financial_advisor | llm | StrOutputParser()


In [ ]:
medical_advisor_prompt = """You are a Medical advisor having more than 15 yeras of consulting and advisory in Medical domain. 
                your answers are very specific, short and encouraging to the users. 
                For a given user's query, provide a appropriate response to the user.
                query: {query}
                    """
medical_advisor = ChatPromptTemplate.from_template(medical_advisor_prompt)

medical_chain = medical_advisor | llm | StrOutputParser()


In [ ]:
general_advisor_prompt = """You are a career advisor having more than 15 yeras of consulting and advisory. 
                your answers are very specific, short and encouraging to the users. 
                For a given user's query, provide a appropriate response to the user.
                query: {query}
                    """
general_advisor = ChatPromptTemplate.from_template(general_advisor_prompt)

general_chain = general_advisor | llm | StrOutputParser()

In [70]:
# based on the classification, return the right chain to get the final response:

def advisor_router(info):
    if 'tech' in info['classification']:
        return tech_chain
    elif 'medical' in info['classification']:
        return medical_chain
    elif 'financial' in info['classification']:
        return financial_chain
    else:
        return general_chain


In [71]:
full_chain = {'classification': classified_chain, 'query': lambda x : x['query']} | RunnableLambda(advisor_router)

In [73]:
query =  'I have completed my B.Pharma, I want to become a good surgen what should be a path for me.'
response= full_chain.invoke({'query': query})

print(response)

Absolutely! Please go ahead and ask your question, and I’ll provide you with a concise and encouraging response.
